In [1]:
%%time
import pandas as pd

from sklearn.metrics import roc_auc_score, make_scorer, accuracy_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler


from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from xgboost import XGBClassifier
from sklearn.feature_selection import RFE


from time import strftime, strptime, mktime, time, ctime

from sklearn import model_selection

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.svm import LinearSVC

from xgboost import XGBClassifier
import xgboost as xgb

from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier,\
ExtraTreesClassifier, GradientBoostingClassifier, VotingClassifier, RandomForestClassifier

from mlxtend.classifier import StackingClassifier, StackingCVClassifier
from mlxtend.feature_selection import ColumnSelector

from sklearn.pipeline import make_pipeline
import numpy as np

import gc


CPU times: user 564 ms, sys: 2.18 s, total: 2.75 s
Wall time: 412 ms


/home/shj16110/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
%%time
df = pd.read_csv('./train.csv')
print df.shape
test = pd.read_csv('./test.csv')
print test.shape

y = df.pop('Approved')
train_id = df.pop('ID')
test_id = test.pop('ID')

def helper(x):
    try:
        tmp = strptime(x, '%d/%m/%y')
        tmp = mktime(tmp)
    except:
        tmp = 563000400
    return tmp

df['Lead_Creation_Date'] = df['Lead_Creation_Date'].apply(helper)
test['Lead_Creation_Date'] = test['Lead_Creation_Date'].apply(helper)

df['DOB'] = df['DOB'].apply(helper)
test['DOB'] = test['DOB'].apply(helper)



(69713, 22)
(30037, 21)
CPU times: user 2.04 s, sys: 163 ms, total: 2.21 s
Wall time: 2.2 s


### from here

In [30]:
df.dtypes

Gender                                  object
DOB                                    float64
Lead_Creation_Date                     float64
City_Code                               object
City_Category                           object
Employer_Code                           object
Employer_Category1                      object
Employer_Category2                     float64
Monthly_Income                         float64
Customer_Existing_Primary_Bank_Code     object
Primary_Bank_Type                       object
Contacted                               object
Source                                  object
Source_Category                         object
Existing_EMI                           float64
Loan_Amount                            float64
Loan_Period                            float64
Interest_Rate                          float64
EMI                                    float64
Var1                                     int64
Cat                                     object
dtype: object

In [31]:
df['Cat']=0
test['Cat']=1
comb = pd.concat([df, test])
comb.shape

(99750, 21)

In [32]:
for i in comb.select_dtypes('object'):
    comb[i] = pd.factorize(comb[i])[0]

train, test = comb[comb['Cat']==0], comb[comb['Cat']==1]
_, _ = train.pop('Cat'), test.pop('Cat')
print train.shape, test.shape

In [36]:
train = train.fillna(axis=1, value=-1)
test = test.fillna(axis=1, value=-1)


s = StandardScaler()
train = s.fit_transform(train)
test = s.transform(test)

print train.shape, test.shape

(69713, 20) (30037, 20)


In [67]:
%%time
m = lambda *x: [i.shape for i in x]
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.2)
print m(X_train, y_train, X_test, y_test)

[(55770, 20), (55770,), (13943, 20), (13943,)]
CPU times: user 14 ms, sys: 0 ns, total: 14 ms
Wall time: 13.1 ms


In [68]:
%%time
d_train = xgb.DMatrix(X_train, label=y_train)
d_test = xgb.DMatrix(X_test, label=y_test)

CPU times: user 33 ms, sys: 2 ms, total: 35 ms
Wall time: 34.1 ms


In [69]:
RANDOM_STATE = 2018
params = {
    'min_child_weight': 0.9,
#     'eta': 0.1,
    'colsample_bytree': 0.9,
    'max_depth': 15,
    'subsample': 0.7,
    'alpha': 0.3,
    'lambda':0.9,
    'gamma': 0.3,
    'silent': 0,
    'learning_rate':0.002,
    'verbose_eval': True,
    'seed': RANDOM_STATE,
    'objective':"binary:logistic",
#     'eval_metric':'error'
#     'num_class':1
    'n_jobs':24,
    'eval_metric':'auc'
    
}
watchlist = [(d_train, 'train'), (d_test, 'test')]



In [70]:
start=time()

clf = xgb.train(params, d_train, 10000, watchlist, early_stopping_rounds=200, 
                verbose_eval=50,  maximize=True,
               )
                


'!!! used %.2f s'%(time()-start)

[0]	train-auc:0.698455	test-auc:0.672698
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 200 rounds.
[50]	train-auc:0.821372	test-auc:0.805049
[100]	train-auc:0.838395	test-auc:0.811237
[150]	train-auc:0.843732	test-auc:0.816717
[200]	train-auc:0.848736	test-auc:0.817877
[250]	train-auc:0.852033	test-auc:0.819156
[300]	train-auc:0.856563	test-auc:0.8192
[350]	train-auc:0.861357	test-auc:0.819367
[400]	train-auc:0.866165	test-auc:0.820122
[450]	train-auc:0.870318	test-auc:0.821467
[500]	train-auc:0.874819	test-auc:0.821982
[550]	train-auc:0.879728	test-auc:0.822597
[600]	train-auc:0.884195	test-auc:0.823586
[650]	train-auc:0.888818	test-auc:0.824565
[700]	train-auc:0.893833	test-auc:0.825024
[750]	train-auc:0.899233	test-auc:0.825877
[800]	train-auc:0.902819	test-auc:0.826205
[850]	train-auc:0.907542	test-auc:0.827287
[900]	train-auc:0.914019	test-auc:0.829012
[950]	train-auc:0.919797	test-auc:0.829756
[10

'!!! used 109.02 s'

In [63]:
%%time
dtest = xgb.DMatrix(test)

CPU times: user 7 ms, sys: 5 ms, total: 12 ms
Wall time: 11.9 ms


In [64]:
%%time
predb = clf.predict(dtest)


CPU times: user 3.91 s, sys: 2 ms, total: 3.91 s
Wall time: 181 ms


___

In [65]:
submit = pd.concat([pd.DataFrame(test_id.values, columns=['ID']), pd.DataFrame(predb, columns=['Approved'])], axis=1)
submit.shape

(30037, 2)

In [66]:
submit.to_csv('xgb_factor3.csv',index=False)

In [441]:
roc_auc_score?

In [442]:
submit2 = pd.concat([pd.DataFrame(test_id.values, columns=['ID']), pd.DataFrame(pred, columns=['Approved'])], axis=1)
submit2.shape

(30037, 2)

In [449]:
accuracy_score(y_test, [0]*y_test.shape[0])

0.9834325467976762

___

In [276]:
cross_val_score(RandomForestClassifier(n_estimators=50, n_jobs=24), df.iloc[:,:-1], df.iloc[:,-1], cv=5)

array([0.87807502, 1.        , 0.99992828, 1.        , 0.98759145])

In [76]:
fr = RandomForestClassifier(n_estimators=50)
fr

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [77]:
fr.fit(df.iloc[:,:-1], df.iloc[:,-1])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [ ]:
fr.predict_log_proba